# CLASIFICACIÓN DE EMOCIONES UTILIZANDO BASE DE DATOS DEAP

## Organización de las bases de datos

In [1]:
#Librerías Necesarias Para el Código organizadas por tipo de llamado y orden alfabético.
import csv
#import h5py
import mne
import scipy
import statistics
import time
import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
import scipy.io as sio
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from scipy import signal
from scipy import spatial
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import LeaveOneOut
from sklearn import svm
from ckaweightedMKL1 import ckaweightedK
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score as acc
from sklearn.utils.class_weight import compute_class_weight
#from unipath import Path


(10, 39, 39) (39, 1)
     pcost       dcost       gap    pres   dres
 0:  7.5304e-01 -3.0348e-01  1e+00  6e-17  4e+00
 1:  7.4936e-01  6.8717e-01  6e-02  1e-16  2e-01
 2:  7.1453e-01  6.9393e-01  2e-02  5e-17  8e-16
 3:  7.1026e-01  7.0811e-01  2e-03  1e-16  6e-16
 4:  7.0930e-01  7.0925e-01  4e-05  1e-16  4e-16
 5:  7.0927e-01  7.0927e-01  4e-07  2e-16  4e-16
Optimal solution found.


## CARGA DE LAS CARACTRERÍSTICAS Y LAS ETIQUETAS BINARIZADAS.

In [2]:
np.set_printoptions(threshold=np.inf) #Imprime todos los valores de los arrays, sirve para visualizar mejor los datos


import pickle
File= open('Wavelets_Excombatants/y.pckl', 'rb')
label_matrix= pickle.load(File)
File.close()

num_valences= 3
num_rois = 8
num_channels= 62
num_trials= 50
#To assess the performance of the C in each emotion, later to evaluate is not necessary.





In [3]:
print(label_matrix.shape)

(50, 1)


In [4]:
def MixKer(K,eta):
    Kn= np.zeros_like((K[1]))
    for i in range(len(eta)):
        Kn= Kn + eta[i]*(K[i])
    return(Kn)

def KerRBF(X):
    d= spatial.distance.pdist(X)
    gamma= 1/(X.shape[1]*X.var())
    K= -spatial.distance.squareform(d)*gamma
    K= np.exp(K)
    return(K)
    
def KerLin(X):
    K = np.matmul(X,X.T)
    return(K)

def graficar(F1_test,Subjects,emocion,save):  
    Bandas=np.arange(1,5)
    Subject= np.arange(1,Subjects+1)
    contador=1
   

    plt.figure(contador,figsize=(6,5))
    plt.title('MKL Emocion: ')
    plt.bar(Subject, F1_test[:], align='center', alpha=0.5, color=['black', 'red', 'green', 'blue', 'cyan'])
    plt.ylim(0,1)
    plt.xlabel('Frequency Band')
    plt.ylabel('F1 Score')
    if save== 'y':
        plt.savefig('FiguresMKL_5/'+'C2_Improved')
    plt.show()
    contador+=1
    
    
def svc_param_selection(y, nfolds,Kernel_train_Wavelets):
    Cs = 1.**np.arange(-1,1)
    #gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C':Cs}
    clf=svm.SVC(kernel='precomputed',class_weight = 'balanced')
    clf.fit(Kernel_train_Wavelets,y)
    grid_search = GridSearchCV(clf,param_grid,  cv=nfolds)
    grid_search.fit(Kernel_train, y)
    grid_search.best_params_
    return grid_search.best_params_



def Moda(N_Emociones,Subjects,Best_C_Matrix):
    Mode_C=np.zeros([N_Emociones])
    Mode_aux=np.zeros([Subjects,N_Emociones])
    Mode_aux2=np.zeros([Subjects,N_Emociones])
    for emo in range(0,N_Emociones):
        print('Emocion: ',emo)
        for sub in range(Subjects):
 
            a= Best_C_Matrix[sub,emo,:]
            a=a.tolist()
            Mode_aux[sub,emo]= max(a,key=a.count)           
      
        try:
            c= Mode_aux[:,emo]
            c=c.tolist()
            Mode_C[emo]=max(c,key=c.count)
        except:
            print('Nada3')
    
    print(' Mode Valence: ',Mode_C[0],'\n Mode Arousal: ',Mode_C[1],'\n Mode Dominance: ',Mode_C[2], '\n Mode Liking: ',Mode_C[3])
    return Mode_C



# SVM RBF Multiple Kernel Learning MKL

## Classification using 3 valences and 5 bands: 
### Here it was performed a classification for each valence and it was combined the ROIs and statistical features in a single dimension, keeping the bands dimension. 

a = np.ones(31)
b = np.ones(19)*-1
c = np.concatenate((a,b))
#print(c)
np.random.shuffle(c)
label_matrix = c
label_matrix.shape = [50,1]
label_matrix.shape

In [ ]:
Csr = 0.1**np.arange(-10,10)
Csr[10]

In [5]:
import warnings
from sklearn.model_selection import KFold as KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
Valences=3
CrossVal= KFold(n_splits=25,shuffle=False)
N_Band= 5
N_Trials= 50
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
#N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs/Features_Stats_Wavelets_'+wave_name[wave]+'_valences_bands'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,1,:,:]))
    
    F1_test_fold = np.zeros([N_folds,Valences])
    ACC_test_fold = np.zeros([N_folds,Valences])
    F1_test = np.zeros([Valences])
    ACC_test = np.zeros([Valences])
    STD_F1_test = np.zeros([Valences])
    STD_ACC_test = np.zeros([Valences])
    #SVM_Prediction_test=np.zeros([Valences,N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([Valences,N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    
    for valence in range(Valences):
        y_test = []
        print('Analizando Wavelet: '+wave_name[wave]+' Valencia : '+str(valence)) 

        for band in range(0,N_Band):
            scaler = MinMaxScaler()
            K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[valence,band,:,:]))
            K[band,:,:] = scaler.fit_transform(K[band,:,:])
            K[band,:,:] = normalize(K[band,:,:])

        fold = 0

        for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[valence,1,:,:])):
            ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
            ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
            ytrain2=np.expand_dims(ytrainMKL,axis=1)
            y_test.append(ytest)

            #Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[valence,band,train_index,:]), (Features_Stats_Wavelets[valence,band,test_index,:])

        #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

            Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
            Ktrain =  K[:,train_index,:]
            Ktrain = Ktrain[:,:,train_index]
            Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
            Ktest = K[:,test_index,:]
            Ktest = Ktest[:,:,train_index]


        #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

            #earlystop = EarlyStopping(
            #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
            #            patience=5,     #max number of acceptable negative steps
            #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
            #            metric='auc',   #the metric we monitor
            #            )

            #scheduler = ReduceOnWorsening()

            #mkl = GRAM(
            #max_iter=1000,          
            #learning_rate=.01,      
            #callbacks=[earlystop],
            #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
            #print(mkl)
            #mkl.predict(Ktest)


        #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

            clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
            #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
            weights_train = clf_Linear_Easy.weights
            #print(weights_train)

            #weights_train = ckaweightedK(Ktrain,ytrain)

            #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
            #weights_train = weights_train/np.sum(weights_train)

            #aux=np.ones(len(weights_train))
            #weights_train = aux-weights_train
            #print(weights_train)
            weights[valence,fold]=weights_train
            Kernel_train= MixKer(Ktrain,weights_train)
            Kernel_test= MixKer(Ktest,weights_train)

        #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
            """"
            clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
            #print ('done')
            #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
            K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
            ##print(K_average)
            #print ('training EasyMKL...', end='')
            clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
            #print ('done')
            #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
            K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
            #print(K_Linear_average)
            Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
            Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
            """
        #---F1-score evaluation for training set------------------------------------------------------------------------------------

            #Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
            #Median= statistics.median(Distances)
            Parameters_grid = svc_param_selection(ytrain,5,Kernel_train)
            SVM_Classifier= svm.SVC(kernel='precomputed',C=Parameters_grid['C'],class_weight = 'balanced')
            SVM_Classifier.fit(Kernel_train,ytrain)
            SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
            SVM_Prediction_test= SVM_Classifier.predict(Kernel_test)
            F1_test_fold[fold,valence] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
            ACC_test_fold[fold,valence]= acc(y_test[fold].T[0],SVM_Prediction_test)
            print("y_test:",y_test[fold].T[0],"SVM:",SVM_Prediction_test,"ACC:",ACC_test_fold[fold,valence],"F1:",F1_test_fold[fold,valence],"C:",Parameters_grid['C'])
            #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


            fold+=1

        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        F1_test[valence] = np.mean(F1_test_fold[:,valence])
        ACC_test[valence]= np.mean(ACC_test_fold[:,valence])
        STD_F1_test[valence] = np.std(F1_test_fold[:,valence])
        STD_ACC_test[valence] = np.std(ACC_test_fold[:,valence])
        print(ACC_test[valence])
    #graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_valences_bands_easy_MKL'
    #Pickle Save F1 Score
    path_save= "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open(path_save+'Weights_'+wav+'.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open(path_save+'Weights_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

(3, 5, 50, 64)
Analizando Wavelet: sym8 Valencia : 0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
y_test: [1. 1.] SVM: [ 1. -1.] ACC: 0.5 F1: 0.6666666666666666 C: 1.0
y_test: [1. 1.] SVM: [ 1. -1.] ACC: 0.5 F1: 0.6666666666666666 C: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
y_test: [1. 1.] SVM: [-1. -1.] ACC: 0.0 F1: 0.0 C: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
y_test: [1. 1.] SVM: [-1.  1.] ACC: 0.5 F1: 0.6666666666666666 C: 1.0
y_test: [ 1. -1.] SVM: [1. 1.] ACC: 0.5 F1: 0.6666666666666666 C: 1.0
y_test: [-1. -1.] SVM: [-1. -

## Classification using 3 valences and 40 bands: 
### Here it was performed a classification for each valence and it was combined the bands and ROIs in a single dimension. 

In [6]:
import warnings
from sklearn.model_selection import KFold as KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
Valences=3
CrossVal= KFold(n_splits=25,shuffle=False)
N_Band= 40
N_Trials= 50
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
#N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs/Features_Stats_Wavelets_'+wave_name[wave]+'_valences_bandrois'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,1,:,:]))
    
    F1_test_fold = np.zeros([N_folds,Valences])
    ACC_test_fold = np.zeros([N_folds,Valences])
    F1_test = np.zeros([Valences])
    ACC_test = np.zeros([Valences])
    STD_F1_test = np.zeros([Valences])
    STD_ACC_test = np.zeros([Valences])
    #SVM_Prediction_test=np.zeros([Valences,N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([Valences,N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    
    for valence in range(Valences):
        y_test = []
        print('Analizando Wavelet: '+wave_name[wave]+' Valencia : '+str(valence)) 

        for band in range(0,N_Band):
            scaler = MinMaxScaler()
            K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[valence,band,:,:]))
            K[band,:,:] = scaler.fit_transform(K[band,:,:])
            K[band,:,:] = normalize(K[band,:,:])

        fold = 0

        for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[valence,1,:,:]),label_matrix):
            ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
            ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
            ytrain2=np.expand_dims(ytrainMKL,axis=1)
            y_test.append(ytest)

            #Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[valence,band,train_index,:]), (Features_Stats_Wavelets[valence,band,test_index,:])

        #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

            Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
            Ktrain =  K[:,train_index,:]
            Ktrain = Ktrain[:,:,train_index]
            Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
            Ktest = K[:,test_index,:]
            Ktest = Ktest[:,:,train_index]


        #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

            #earlystop = EarlyStopping(
            #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
            #            patience=5,     #max number of acceptable negative steps
            #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
            #            metric='auc',   #the metric we monitor
            #            )

            #scheduler = ReduceOnWorsening()

            #mkl = GRAM(
            #max_iter=1000,          
            #learning_rate=.01,      
            #callbacks=[earlystop],
            #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
            #print(mkl)
            #mkl.predict(Ktest)


        #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

            clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
            #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
            weights_train = clf_Linear_Easy.weights
            #print(weights_train)

            #weights_train = ckaweightedK(Ktrain,ytrain)

            #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
            #weights_train = weights_train/np.sum(weights_train)

            #aux=np.ones(len(weights_train))
            #weights_train = aux-weights_train
            #print(weights_train)
            weights[valence,fold]=weights_train
            Kernel_train= MixKer(Ktrain,weights_train)
            Kernel_test= MixKer(Ktest,weights_train)

        #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
            """"
            clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
            #print ('done')
            #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
            K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
            ##print(K_average)
            #print ('training EasyMKL...', end='')
            clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
            #print ('done')
            #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
            K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
            #print(K_Linear_average)
            Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
            Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
            """
        #---F1-score evaluation for training set------------------------------------------------------------------------------------

            #Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
            #Median= statistics.median(Distances)
            Parameters_grid = svc_param_selection(ytrain,5,Kernel_train)
            SVM_Classifier= svm.SVC(kernel='precomputed',C=Parameters_grid['C'],class_weight = 'balanced')
            SVM_Classifier.fit(Kernel_train,ytrain)
            SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
            SVM_Prediction_test= SVM_Classifier.predict(Kernel_test)
            F1_test_fold[fold,valence] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
            ACC_test_fold[fold,valence]= acc(y_test[fold].T[0],SVM_Prediction_test)
            print("y_test:",y_test[fold].T[0],"SVM:",SVM_Prediction_test,"ACC:",ACC_test_fold[fold,valence],"F1:",F1_test_fold[fold,valence])
            #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


            fold+=1

        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        F1_test[valence] = np.mean(F1_test_fold[:,valence])
        ACC_test[valence]= np.mean(ACC_test_fold[:,valence])
        STD_F1_test[valence] = np.std(F1_test_fold[:,valence])
        STD_ACC_test[valence] = np.std(ACC_test_fold[:,valence])
        print(F1_test[valence])
    #graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_valences_bandrois_easy_MKL'
    #Pickle Save F1 Score
    path_save= "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open(path_save+'Weights_'+wav+'.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open(path_save+'Weights_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

(3, 40, 50, 8)
Analizando Wavelet: sym8 Valencia : 0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [ 1. -1.] ACC: 0.5 F1: 0.6666666666666666
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [-1.  1.] ACC: 0.5 F1: 0.6666666666666666
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0
y_test: [ 1. -1.] SVM: [-1.  1.] ACC: 0.0 F1: 0.0
y_test: [-1. -1.] SVM: [-1. -1.] ACC: 1.0 F1: 1.0
y_test: [-1. -1.] SVM: [-1. -1.] ACC: 1.0 F1: 1.0
y_test: [-1. -1.] SVM: [-1. -1.] ACC: 1.0 F1: 1.0
y_test: [-1. -1.] SVM: 

## Classification using 5 bands and no-valences: 
### Here it was performed a classification mixing the Valences, ROIs and statistical features in a single dimension, keeping the bands dimension. 

In [7]:
import warnings
from sklearn.model_selection import KFold as KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
Valences=3
CrossVal= KFold(n_splits=25,shuffle=False)
N_Band= 5
N_Trials= 50
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
#N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs/Features_Stats_Wavelets_'+wave_name[wave]+'_valencesrois'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    F1_test_fold = np.zeros([N_folds])
    ACC_test_fold = np.zeros([N_folds])
    #SVM_Prediction_test=np.zeros([Valences,N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    

    y_test = []
    print('Analizando Wavelet: '+wave_name[wave]) 

    for band in range(0,N_Band):
        scaler = MinMaxScaler()
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))
        K[band,:,:] = scaler.fit_transform(K[band,:,:])
        K[band,:,:] = normalize(K[band,:,:])

    fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:]),label_matrix):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test.append(ytest)

        #Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[valence,band,train_index,:]), (Features_Stats_Wavelets[valence,band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

        clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
        weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        #weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        #Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        #Median= statistics.median(Distances)
        Parameters_grid = svc_param_selection(ytrain,5,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test= SVM_Classifier.predict(Kernel_test)
        F1_test_fold[fold] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
        ACC_test_fold[fold]= acc(y_test[fold].T[0],SVM_Prediction_test)
        print("y_test:",y_test[fold].T[0],"SVM:",SVM_Prediction_test,"ACC:",ACC_test_fold[fold],"F1:",F1_test_fold[fold],"C:",Parameters_grid['C'])
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        F1_test = np.mean(F1_test_fold[:])
        ACC_test= np.mean(ACC_test_fold[:])
        STD_F1_test = np.std(F1_test_fold[:])
        STD_ACC_test = np.std(ACC_test_fold[:])
        print(ACC_test)
    #graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_valencesrois_easy_MKL'
    #Pickle Save F1 Score
    path_save= "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open(path_save+'Weights_'+wav+'.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open(path_save+'Weights_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

(5, 50, 192)
Analizando Wavelet: sym8
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.04
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.08
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.12
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.16
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.2
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.24
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.28
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.32
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.36
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.4
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.44
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.48
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.52
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
I

## Classification using 40 bands and no-valences: 
### Here it was performed a classification mixing the Valences, ROIs and statistical features in a single dimension, keeping the bands dimension. 

In [8]:
import warnings
from sklearn.model_selection import KFold as KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
Valences=3
CrossVal= KFold(n_splits=25,shuffle=False)
N_Band= 40
N_Trials= 50
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
#N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs/Features_Stats_Wavelets_'+wave_name[wave]+'_bandrois'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    F1_test_fold = np.zeros([N_folds])
    ACC_test_fold = np.zeros([N_folds])
    #SVM_Prediction_test=np.zeros([Valences,N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    

    y_test = []
    print('Analizando Wavelet: '+wave_name[wave]) 

    for band in range(0,N_Band):
        scaler = MinMaxScaler()
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))
        K[band,:,:] = scaler.fit_transform(K[band,:,:])
        K[band,:,:] = normalize(K[band,:,:])

    fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:]),label_matrix):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test.append(ytest)

        #Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[valence,band,train_index,:]), (Features_Stats_Wavelets[valence,band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

        clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
        weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        #weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        #Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        #Median= statistics.median(Distances)
        Parameters_grid = svc_param_selection(ytrain,5,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test= SVM_Classifier.predict(Kernel_test)
        F1_test_fold[fold] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
        ACC_test_fold[fold]= acc(y_test[fold].T[0],SVM_Prediction_test)
        print("y_test:",y_test[fold].T[0],"SVM:",SVM_Prediction_test,"ACC:",ACC_test_fold[fold],"F1:",F1_test_fold[fold],"C:",Parameters_grid['C'])
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        F1_test = np.mean(F1_test_fold[:])
        ACC_test= np.mean(ACC_test_fold[:])
        STD_F1_test = np.std(F1_test_fold[:])
        STD_ACC_test = np.std(ACC_test_fold[:])
        print(ACC_test)
    #graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_bandrois_easy_MKL'
    #Pickle Save F1 Score
    path_save= "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open(path_save+'Weights_'+wav+'.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open(path_save+'Weights_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

(40, 50, 24)
Analizando Wavelet: sym8
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.04
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.08
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.12
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.16
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.2
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.24
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.28
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.32
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.36
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.4
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.44
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.48
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.52
y_test: [1. 1.] SVM: [1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
I

In [9]:
print(ACC_test)
print(STD_F1_test)
print(weights)

0.98
0.06531972647421809
[[0.01588725 0.02603322 0.01255076 0.01742709 0.04047021 0.03661889
  0.01475814 0.01573334 0.0196073  0.03442284 0.03346093 0.02581269
  0.00983011 0.04265883 0.03256541 0.030588   0.02293819 0.03663094
  0.02054358 0.03953424 0.02803212 0.01317314 0.00971336 0.03065839
  0.02521009 0.03553157 0.01827888 0.03335074 0.01812193 0.0327027
  0.02194544 0.01875693 0.0117016  0.02918493 0.03263361 0.0347648
  0.01530426 0.00878399 0.02045565 0.03362393]
 [0.01490839 0.02683937 0.01295598 0.01699162 0.03723336 0.03393349
  0.0175431  0.01676833 0.01845027 0.03475902 0.03138633 0.0257514
  0.01015534 0.03994963 0.0268482  0.03309152 0.03029085 0.03196323
  0.0196741  0.04013993 0.03127073 0.01364894 0.01073404 0.03148556
  0.02363907 0.04195682 0.01962914 0.03330971 0.01867045 0.03384001
  0.02161804 0.01742275 0.01162579 0.02817327 0.03149624 0.03512121
  0.01341665 0.00890024 0.02216689 0.03224098]
 [0.01456757 0.02522033 0.01391075 0.01797504 0.03863091 0.03508789


# UNUSED

# 1)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
CrossVal= LeaveOneOut()
N_Band= 5
N_Trials= 50
N_Rois = 8
C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs/Features_Stats_Wavelets_'+wave_name[wave]+'_valencesrois'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    
    SVM_Prediction_test=np.zeros([N_folds])
    y_test = np.zeros([N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
  
    print('Analizando Wavelet: '+wave_name[wave]+' Valencia : ') 

    for band in range(0,N_Band):
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))

        fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:])):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        y_test[fold] = ytest
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test[fold] = ytest

        Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[band,train_index,:]), (Features_Stats_Wavelets[band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

        clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
        weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        #weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        Median= statistics.median(Distances)
        #Gamma= 1/(2*(Median**2))
        Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
        Parameters_grid = svc_param_selection(ytrain,5,Gamma,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',gamma=Gamma,C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test[fold]= SVM_Classifier.predict(Kernel_test)
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

    print('INICIO TEST')
    #Calcular F1_test
    #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
    F1_test = f1_score(y_test[:],SVM_Prediction_test[:])
    ACC_test= acc(y_test[:],SVM_Prediction_test[:])
    print(F1_test)
    #graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_valencesrois_easy'
    #Pickle Save F1 Score
    import pickle
    file= open('Wavelets_Results_ROIs/F1_Score_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open('Wavelets_Results_ROIs/F1_Score_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open('Wavelets_Results_ROIs/ACC_Score_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open('Wavelets_Results_ROIs/ACC_Score_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open('Wavelets_Results_ROIs/Weights_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open('Wavelets_Results_ROIs/Weights_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

# 2)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
CrossVal= LeaveOneOut()
N_Band= 40
N_Trials= 50
C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs/Features_Stats_Wavelets_'+wave_name[wave]+'_bandrois'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    SVM_Prediction_test=np.zeros([N_folds])
    y_test = np.zeros([N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    
   
    print('Analizando Wavelet: '+wave_name[wave]) 

    for band in range(0,N_Band):
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))

        fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:])):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        y_test[fold] = ytest
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test[fold] = ytest

        Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[band,train_index,:]), (Features_Stats_Wavelets[band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

        clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
        weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        #weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        Median= statistics.median(Distances)
        Parameters_grid = svc_param_selection(ytrain,5,Gamma,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',gamma=Gamma,C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test[fold]= SVM_Classifier.predict(Kernel_test)
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

    #---F1-score evaluation for test set---------------------------------------------------------------------------------------

    print('INICIO TEST')
    #Calcular F1_test
    #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
    F1_test = f1_score(y_test[:],SVM_Prediction_test[:])
    ACC_test = acc(y_test[:],SVM_Prediction_test[:])
    print(F1_test)
#graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_bandrois_easy'
    #Pickle Save F1 Score
    import pickle
    file= open('Wavelets_Results_ROIs/F1_Score_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open('Wavelets_Results_ROIs/F1_Score_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open('Wavelets_Results_ROIs/ACC_Score_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open('Wavelets_Results_ROIs/ACC_Score_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open('Wavelets_Results_ROIs/Weights_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open('Wavelets_Results_ROIs/Weights_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

In [ ]:
weights_train

In [ ]:
File= open('Wavelets_Results/F1_Score_'+'bior3.3_sparse'+'_MKL.pckl', 'rb')
F1_test = pickle.load(File)
File.close()
print(F1_test)

In [ ]:
#F1_test.tofile('Test_5\F1_Test_KRBF_C3_MKL_Improved'+'.txt')
#Best_C_Matrix.tofile('Test_5\Best_C_Matrix_C2_MKL'+'.txt')

# GRÁFICA DEL F1_SCORE PARA CADA SUJETO EN CADA BANDA Y PARA CADA C 


In [ ]:
import warnings
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
Subjects= 32
CrossVal= LeaveOneOut()
N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,1,:,:]))
N_Band= 6
N_Trials= 40
N_Emociones=4
C = 2.**np.arange(-15,19)
N_cs = len(C)
Best_C_Matrix=np.zeros([Subjects,N_Emociones,N_folds])
F1_train = np.zeros([Subjects,N_Emociones,N_folds,N_cs])
F1_test = np.zeros([Subjects,N_Emociones])
SVM_Prediction_test=np.zeros([Subjects,N_Emociones,N_folds])
y_test = np.zeros([Subjects,N_folds])
K = np.zeros([N_Band,N_Trials,N_Trials])

#---Iterations for each Emotion and subject--------------------------------------------------------------------------

for emo in range(N_Emociones):
    for sub in range(Subjects):
        print('Análisis Sujeto: ',sub, 'Emoción: ',emo)
        if (sub==26 and emo==2):
            sub+=1
            
        for band in range(0,N_Band):
            K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[sub,band,:,:]))

        fold = 0
        emocion=emo
        for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[sub,1,:,:])):
            ytrain, ytest = label_threshold_matrix_5[sub,train_index,emocion], label_threshold_matrix_5[sub,test_index,emocion]
            y_test[sub,fold] = ytest
            ytrainMKL, ytestMKL = label_matrix[sub,train_index,emocion], label_matrix[sub,test_index,emocion]
            ytrain2=np.expand_dims(ytrainMKL,axis=1)
            y_test[sub,fold] = ytest

            Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[sub,band,train_index,:]), (Features_Stats_Wavelets[sub,band,test_index,:])

        #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

            Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
            Ktrain =  K[:,train_index,:]
            Ktrain = Ktrain[:,:,train_index]
            Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
            Ktest = K[:,test_index,:]
            Ktest = Ktest[:,:,train_index]

        #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

            #print(Ktrain.shape, ytrain2.shape)
            weights_train = ckaweightedK(Ktrain,ytrain2)
            #print(weights_train)
            Kernel_train= MixKer(Ktrain,weights_train)
            Kernel_test= MixKer(Ktest,weights_train)

        #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
            """"
            clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
            #print ('done')
            #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
            K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
            ##print(K_average)
            #print ('training EasyMKL...', end='')
            clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
            #print ('done')
            #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
            K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
            #print(K_Linear_average)
            Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
            Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
            """
        #---F1-score evaluation for training set------------------------------------------------------------------------------------

            Best_C= -10000
            Best_F1=0
            for c in range(len(C)):
                Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
                Median= statistics.median(Distances)
                Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
                SVM_Classifier= svm.SVC(kernel='precomputed',C=C[c])
                SVM_Classifier.fit(Kernel_train,ytrain)
                SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
                SVM_Prediction_test[sub,emo,fold]= SVM_Classifier.predict(Kernel_test)
                F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')

                if F1_train[sub,emo,fold,c]>Best_F1:
                    Best_C_Matrix[valence,fold]=C[c]
                    Best_C= C[c]
                    Best_F1=F1_train[valence,fold,c]
            fold+=1



        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        F1_test[sub,emo] = f1_score(y_test[sub,:],SVM_Prediction_test[sub,emo,:],average='macro')
        #print(F1_test[sub,band,c])
        #time.sleep(4)
    graficar(F1_test,Subjects,emocion)
                

In [ ]:
%matplotlib notebook
graficar(F1_test,Subjects,3)
Moda(N_Emociones,Subjects,Best_C_Matrix)

In [ ]:
#F1_test.tofile('Test\F1_Test_Customlib_RBF_Lineal_C3_'+Emocion[emocion]+'.txt')

In [ ]:
band=['Theta','Low Alpha', 'High Alpha', 'Alha', 'Betha']


for i in range(0,7):
    plt.figure(i)
    #plt.title('Kernel for '+ band[i])
    plt.imshow(Ktrain[i],cmap= plt.cm.get_cmap('OrRd'))
    #plt.savefig('DEAP_Figures/Kernel_'+str(i))

In [ ]:
plt.imshow(Kernel_train,cmap= plt.cm.get_cmap('OrRd'))
#plt.savefig('DEAP_Figures/Final_Kernel')
print(weights_train) 


In [ ]:
ensayo=np.matmul(ytrain2,ytrain2.T)
plt.imshow(ensayo,cmap= plt.cm.get_cmap('OrRd'))

In [ ]:
%matplotlib notebook
x=np.arange(0,496)
y=np.arange(0,40)
X,Y= np.meshgrid(x,y)
z=Y*X
print(Feature_MSC_Matrix_C2[0,0].shape)
fig= plt.figure()
ax= fig.gca(projection='3d')
ax.contour(X,Y,Feature_MSC_Matrix_C2[0,0,:,:])